# 02 Interactive Dashboards

> "Above all else show the data." ~ Edward Tufte

![img](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Sydney_Australia._%2821339175489%29.jpg/1200px-Sydney_Australia._%2821339175489%29.jpg)

## Table of Contents

0. The Dashboard
1. Scenario
2. Use Cases
3. The Data
4. Top-Down Dashboard Breakdown
5. Putting it all Together
6. Summary

In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import holoviews as hv, panel as pn
from holoviews import dim, opts
import geopandas as gpd, geoviews as gv
from holoviews.element import tiles
import IPython

hv.extension('bokeh', 'matplotlib')

pn.extension()

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

## 00. The Dashboard

There is an HTML copy of the dashboard available in a folder called, **dashboards**, and the following command run and display the dasboard in your notebook.

In [ ]:
# IPython.display.HTML('dashboards/interactive_dash.html')

## 01. Scenario

Imagine we were planning our next vacation and wanted to head to the land down under. Sydney, as 
it is well known, is one of the most picturesque cities in the world, and that also makes it an 
expensive one. Because of this, we will be examining the variation of prices for a listing across 
different measures including median price per suburb, average amount of reviews per suburb and property type, etc.

## 02. Use Cases

These data can be useful not only for those planning a vacation but also for

1. Tourism agents
2. Data analysts in areas such as government of real estate
3. Restaurants wanting to cater to tourists
4. Real estate agents
5. Many more

## 03. The Data

The data we will be using for our dashboard on this notebook comes from Airbnb and it contains 
information about all of the listings available in the metropolitan area of Sydney up until April 
2020. In addition, most of the information about a listing displayed on Airbnb and in this dataset 
may still be found on the website as it was a scraped directly from the it.

There are some additional variables such as
- `min_price_per_stay` - Airbnb provides prices per night but hosts are able to set a minimum amount 
of nights for which you can book their place. As such, the amount available is not always indicative 
of the amount to be paid and thus, this variable shows `price * minimum_nights + cleaning_fee`
- `two_weeks_price` - same formula as above but istead of minimum nights the price has been multiplied by 14

Lastly, we also have a geodataframe containing the coordinates of the major suburbs in Sydney.

### The Airbnb Data

In [ ]:
df = pd.read_parquet(os.path.join('..', 'data', 'interactive', "sydney_airbnb.parquet"))
# df = pd.read_csv(os.path.join('..', 'data', 'interactive', "sydney_airbnb.csv"))
print(df.shape)

In [ ]:
# shows the first 5 rows of the dataframe
df.head()

In [ ]:
# show the amount of rows, columns, missing values, and the data types of each variable
df.info()

### The GeoDataFrame

We will combine our listings data with some geographical data about Sydney to create a map later on.

In [ ]:
subs_geo = gpd.read_file(os.path.join('..', 'data', 'interactive', "neighbourhoods.geojson"))

In [ ]:
# shows the first 5 rows of the dataframe
subs_geo.head()

In [ ]:
# show the amount of rows, columns, missing values, and the data types of each variable
subs_geo.info()

## 04. Top-Down Dashboard Breakdown

This section is all about deconstructing our dashboard. We will start with the widgets, the tools that provide us with interactivity, and work our way backwards until we get to the final output.

### The Widgets

Panel contains plenty of widgets for us from a function called `interact` and another called `widgets`. Interact 
provides interactive capabilities to the parameters of different kinds of plots that have been created by 
functions. The `interact` function can create a user interface as tiny as input box and as complex as a full app.

In contrast, widgets provide a finer control on the interactivity behind the scenes of a complex app. They also 
allows you to use changing parameters of declared functions inside the decorator, `p.depends`, which will create 
an interactive chart for us.

Let's first create a list will all of the properties in our dataset, and then check out how widgets work.

In [ ]:
property_types = list(df['property_type'].unique())
property_types

In [ ]:
p_type = pn.widgets.Select(value='Apartment', options=property_types, name='Property Type')
p_type

In [ ]:
p_type.value

Noticed above using `pn.widgets.Select` we, assigned a default `value` to a widget from 
which our function will have a list of `options` to choose from, and we also gave it a name.

Before we try another one, let's check the review scores for hosts around Sydney.

In [ ]:
df.review_scores_rating.unique()

Let's create a categorical variable similar to what in marketing is considered the net promoter 
score. Our variable will group reviews of `>=90` into one bin, `>=70` into another, and the rest 
into another bucket. These buckets will be called `Excellent`, `Okay`, and `No Bueno`, respectively.

In [ ]:
# we will start with a column saying no reviews
df['nps'] = 'None'

In [ ]:
# then we will create a mask for those listings with at least 1 review
revs_mask = df['number_of_reviews'] > 0

In [ ]:
# we will create a function to bin our groups
def get_nps(x):
    if x >= 90:
        return "Excellent"
    elif x >= 70:
        return "Okay"
    else:
        return "No Bueno"

Using the locator selector, we will add the new values of our function back into our new `nps` column.

In [ ]:
df.loc[revs_mask, 'nps'] = df.loc[revs_mask, 'review_scores_rating'].apply(get_nps)
df['nps'].head(10)

In [ ]:
# let's evaluate the total count for each group
df['nps'].value_counts()

Let's create a list for our new widget and remove the `None` from it as we are only interested in those with a review.

In [ ]:
nps_vals = list(df.nps.unique())
nps_vals.remove('None')

Now we can put together another widget.

In [ ]:
nps = pn.widgets.Select(value='Excellent', options=nps_vals, name='Net Promoters')
nps

While we will only use the `pn.widgets.Select` and `pn.interact` functions for this notebook, there are other amazing widgets that you should definitely explore at [panel widgets](https://panel.holoviz.org/user_guide/Widgets.html).

### Exercise

1. Pick any categorical column from the dataset.
2. Get the unique values of such category.
3. Create a widget, assign it to a variable called `my_widget`, and display it.

### The Map

We will use a map to guide us into where to find the best priced locations for us to stay at. Let's examine the prices of apartment listing per suburb. We will filter for apartments, group by the neighborhood, and then take the median price of the minimum price per stay column.

In [ ]:
df.shape

In [ ]:
data = df[df['property_type'] == 'Apartment'].copy()
data.shape

In [ ]:
data_group = data.groupby('neighbourhood_cleansed')['min_price_per_stay'].min().reset_index()
data_group.head()

We will need to merge our dataset with the geodataframe containing the neighborhoods.

In [ ]:
subs_geo.head()

In [ ]:
data_merged = (
    data_group.merge(
        subs_geo[['neighbourhood', 'geometry']], left_on='neighbourhood_cleansed', right_on='neighbourhood'
    ).drop('neighbourhood', axis=1).rename(columns={'neighbourhood_cleansed': "Suburb"})
)
data_merged.head()

In [ ]:
type(data_merged)

Let's make that panda into a geopandas dataframe again.

In [ ]:
geo_data = gpd.GeoDataFrame(data_merged)
type(geo_data)

Let's now create our choropleth map.

In [ ]:
geo_fig = gv.Polygons(
    geo_data, vdims=['Suburb', 'min_price_per_stay']
).opts(
    tools=['hover'], width=500, height=400, color='min_price_per_stay', cmap='viridis_r',
    colorbar=True, toolbar='below', xaxis=None, yaxis=None, color_levels=20
)

Before we visualize it, let's overlay a map behind it.

In [ ]:
(tiles.CartoLight() * geo_fig).relabel(label=f'Median Listing Price by Suburb')

Fantastic! Now, the key thing to do to create interactive widgets is to package our visualizations 
into functions and pass our widget from before through the `pn.depends` decorator as 
`@pn.depends(p_type.param.value)`. We need to remember to tell `panel` which parameter value to access 
from our widget.

So now, let's use the same lines from above for our function.

In [ ]:
@pn.depends(p_type.param.value)
def get_map(p_type, **kwargs):
    
    data = df[df['property_type'] == p_type].copy()
    data_group = data.groupby('neighbourhood_cleansed')['min_price_per_stay'].median().reset_index()
    data_merged = (
        data_group.merge(
            subs_geo[['neighbourhood', 'geometry']], left_on='neighbourhood_cleansed', right_on='neighbourhood'
        ).drop('neighbourhood', axis=1).rename(columns={'neighbourhood_cleansed': "Suburb"})
    )
    
    geo_data = gpd.GeoDataFrame(data_merged)
    fig = gv.Polygons(
        geo_data, vdims=['Suburb', 'min_price_per_stay']
    ).opts(
        tools=['hover'], width=500, height=400, color='min_price_per_stay', cmap='viridis_r',
        colorbar=True, toolbar='above', xaxis=None, yaxis=None, color_levels=20
    )
    return (tiles.CartoLight() * fig).relabel(label=f'Median Listing Price per {p_type}')

In [ ]:
# let's test our function and widget in a row object
pn.Row(p_type, get_map)

Great. We can also transfer the same idea to `pn.interact`. Let's create another grouped dataframe with more descriptive statistics.

In [ ]:
nb_stay_stats = df.groupby('neighbourhood_cleansed')['min_price_per_stay'].agg(['min', 'mean', 'median', 'max', 'count']).reset_index()
nb_stay_stats.head()

In [ ]:
# we need to combine it again with our geodataframe containing the polygos
geo_nb_stats = gpd.GeoDataFrame(
    nb_stay_stats.merge(
        subs_geo[['neighbourhood', 'geometry']], left_on='neighbourhood_cleansed', right_on='neighbourhood'
    ).drop('neighbourhood', axis=1).rename(columns={'neighbourhood_cleansed': "Suburb"})
)
geo_nb_stats.head()

We will create a second function and pass it to `pm.interact` alongside the parameters we want it to interact with, the descriptive columns.

In [ ]:
def get_map2(col='mean', **kwargs):
    return (
        tiles.CartoLight() * (gv.Polygons(
            geo_nb_stats.copy(), vdims=['Suburb', col]
        ).opts(
            tools=['hover'], width=550, height=600, color=col, cmap='viridis_r',colorbar=True,
            toolbar='above', xaxis=None, yaxis=None, color_levels=38))
    )

In [ ]:
pn.interact(get_map2, col=['min', 'mean', 'median', 'max', 'count'])

### Exercise

Create a choropleth map of the median two week stay with an interactive widget containing the cancellation policy. For this you will need to
1. Create a list with the unique values of the cancellation policy column
2. Create a groupeby object by the cancellation policy and the two week stay price
3. Combine the group data with the geodataframe
4. Create a function with your map and cancellation policy
5. Display widget and map as a row or column

### The Table

Tables are exactly that, a view at the data but with an interactive component to it. We will create one for the reviews in our dataset.

In [ ]:
# let's change the names of these columns first
reviews = ['review_scores_checkin', 'review_scores_cleanliness', 'review_scores_accuracy', 
           'review_scores_location', 'review_scores_communication', 'review_scores_value']
new_names = ['Checkin', 'Cleanliness', 'Accuracy', 'Location', 'Communication', 'Value']

names_dict = {old:new for old, new in zip(reviews, new_names)}
names_dict

We'll need two masks, one for the Apartment and another for listings with at least 1 review.

In [ ]:
p_mask = df['property_type'] == 'Apartment'
revs_mask = df['number_of_reviews'] > 0

In [ ]:
# filter our undersired listings
data = df[p_mask & revs_mask].copy()
# rename the review columns
data.rename(names_dict, axis=1, inplace=True)

We will examine the average review score for 6 criteria and add the results to a holoviews table.

In [ ]:
data[new_names].mean().to_frame(name='vals').reset_index()

In [ ]:
data_group = data[new_names].mean().to_frame(name='vals').reset_index()
data_group.columns = ['Reviews', 'Average Score']

In [ ]:
# the table can take other data sources as well
table = hv.Table(data_group).opts(width=250, height=180)
table

Now let's wrap our process again into a function for which the only parameter that can change is the property type.

In [ ]:
@pn.depends(p_type.param.value)
def get_rev_table(p_type, **kwargs):
    
    p_mask = df['property_type'] == p_type
    revs_mask = df['number_of_reviews'] > 0

    data = df[p_mask & revs_mask].copy()    
    data.rename(names_dict, axis=1, inplace=True)
    data_group = data[new_names].mean().to_frame(name='vals').reset_index()
    data_group.columns = ['Reviews', 'Average Score']
    
    return hv.Table(data_group).opts(width=250, height=180, bgcolor='red')

We can also examine the functions individually with one value.

In [ ]:
get_rev_table("Apartment")

In [ ]:
# remember if you need help use
# hv.help(opts.Table)

In [ ]:
pn.Column(p_type, get_rev_table)

### The Whiskers

Box plots are incredibly useful for telling us the key descriptive statistics of a 
variable. Mainly, they provide us with the min, interquartile, median, max, and a 
view of the outliers, if any.

For our purposes, let's assume that we have a budget of 2000 AUD for a 2-week vacation, 
and also that we will only stay at places with excellent ratings.

In [ ]:
nps_mask = df['nps'] == 'Excellent'
budget = df['two_weeks_price'] < 2000

In [ ]:
# create a copy of the filtered dataset
data = df.loc[nps_mask & budget, ['property_type', 'two_weeks_price']].copy()

In [ ]:
# since our label is a bit long we will add it to a variable
label = "(2-Week Stay) Price Range per Property Type with  Reviews"

Let's now create our box plot using holoview function `hv.BoxWhisker`, which contains most of the parameters we are familiar with by now.

In [ ]:
bw = hv.BoxWhisker(data, 'property_type', 'two_weeks_price', label=label)
bw

As we can see, our chart needs some options to be more useful, so let's give some of the ones we have already used.

In [ ]:
bw.opts(box_fill_color='#D5E051', box_line_color='#5F6062', width=600, height=250, box_line_width=1,
        whisker_color='#FFFFFF', xrotation=25, bgcolor='#5F6062',labelled=[], outlier_color='#FFFFFF')

Let's now package eveything in a function that will change given the net promoter score.

In [ ]:
@pn.depends(nps.param.value)
def cat_whisker(nps, **kwargs):
    
    nps_mask = df['nps'] == nps
    budget   = df['two_weeks_price'] < 2000
    data     = df.loc[nps_mask & budget, ['property_type', 'two_weeks_price']].copy()
    label    = f"(2-Week Stay) Price Range per Property Type with {nps} Reviews"
    boxw     = hv.BoxWhisker(data, 'property_type', 'two_weeks_price', label=label)
    
    return boxw.opts(
        box_fill_color='#D5E051', box_line_color='#5F6062', width=600, height=250, box_line_width=1,
        whisker_color='#FFFFFF', xrotation=25, bgcolor='#5F6062', labelled=[], outlier_color='#FFFFFF'
    )

In [ ]:
pn.Column(nps, cat_whisker)

### Exercise

Create a box and whisker plot showing the distribution of the cleaning fee column. You can use the property type as your x axis again.

### Plots/Bars

What we want to know.

> What's the difference between the average number of reviews received per suburb and by super hosts and regular hosts?

By now you probably recognize the pattern we are following. We first declare the widgets we want
our visualizations to interact with, we then filter a dataframe, group by some other measure, and 
then add our widgets to the mix with our chart inside a function. Let's do that same process again.

In [ ]:
# at least 1 review
revs_mask = df['number_of_reviews'] > 0

In [ ]:
data  = df[nps_mask & p_mask & revs_mask].copy()
group = data.groupby(['neighbourhood_cleansed', 'host_is_superhost'])['number_of_reviews'].mean().reset_index()
group.head()

We now have a different interest. We want to know if super hosts ge more reviews, on average, than regular hosts. Let's filter 2 dataframes for this.

In [ ]:
superh   = group[group['host_is_superhost'] == 't']
regularh = group[group['host_is_superhost'] == 'f']

We then create our first scatter and load it with options. Remember, the second scatter does not need as many functions as we will overlay it on top of the sorted first one.

In [ ]:
dots1 = (
    hv.Scatter(
        superh, 'neighbourhood_cleansed', 'number_of_reviews', label='Super Hosts'
    ).sort('number_of_reviews')
     .opts(
         color='#D5E051', width=500, show_grid=True, height=400, invert_axes=True, size=7, tools=['hover'],
         legend_position='bottom_right', toolbar='right', labelled=[], title="Average # of Reviews per Suburb")
)
dots1

In [ ]:
dots2 = hv.Scatter(regularh, 'neighbourhood_cleansed', 'number_of_reviews', label='Regular Hosts').opts(size=7, color='#D8DEE9')
dots2

We can overlay the two with the `*` sign.

In [ ]:
(dots1 * dots2)

Let's wrap it all up in a function now but this time, we will use both parameters, the nps and the property type. Note that you can add as many as you'd like and that can increase the complexity of your visualization and app.

In [ ]:
@pn.depends(p_type.param.value, nps.param.value)
def my_dots(p_type, nps, **kwargs):
    
    nps_mask  = df['nps'] == nps
    p_mask    = df['property_type'] == p_type
    revs_mask = df['number_of_reviews'] > 0
    data      = df[nps_mask & p_mask & revs_mask].copy()
    group     = data.groupby(['neighbourhood_cleansed', 'host_is_superhost'])['number_of_reviews'].mean().reset_index()
    superh    = group[group['host_is_superhost'] == 't']
    regularh  = group[group['host_is_superhost'] == 'f']
    
    dots1 = hv.Scatter(
        superh, 'neighbourhood_cleansed', 'number_of_reviews', label='Super Hosts'
    ).sort(
        'number_of_reviews'
    ).opts(
        color='#D5E051', width=500, show_grid=True, 
        height=400, invert_axes=True, size=7, tools=['hover'], 
        legend_position='bottom_right', toolbar='right', labelled=[], title="Average # of Reviews per Suburb"
    )
    dots2 = hv.Scatter(regularh, 'neighbourhood_cleansed', 'number_of_reviews', label='Regular Hosts').opts(size=7, color='#ff5733')
    return (dots1 * dots2)

In [ ]:
pn.Column(p_type, nps, my_dots)

### The BANs

Let's go immediately to the function of 1 BAN with a title.

In [62]:
@pn.depends(p_type.param.value)
def ban_small(p_type):
    
    p_mask     = df['property_type'] == p_type
    data       = df[p_mask].copy()
    main_title = pn.pane.Markdown(f"# {p_type}", styles={'color':'#FFFFFF'})
    return pn.Column(
        main_title, pn.indicators.Number(
            name='Listings', value=p_mask.sum(), 
            default_color='#FFFFFF', align='start', 
            font_size='13pt', title_size='13pt', format='{value:,.0f}')
    )

In [63]:
pn.Row(p_type, ban_small)

Row
    [0] Select(name='Property Type', options=['Apartment', ...], value='House')
    [1] ParamFunction(function, _pane=Column, defer_load=False)

Let's make the function above a bit more complex.

In [64]:
@pn.depends(p_type.param.value)
def bans(p_type):
    
    g_opts     = dict(default_color='#FFFFFF', align='start', font_size='13pt', title_size='13pt')
    p_mask     = df['property_type'] == p_type
    data       = df[p_mask].copy()    
    listings   = p_mask.sum() 
    super_host = data.groupby('host_is_superhost')['host_is_superhost'].count().loc['t']
    avg_price  = data['price'].mean()
    
    # the title will change depending on the property type
    main_title = pn.pane.Markdown(f"# {p_type}", styles={'color':'#FFFFFF'})
    
    ban1 = pn.indicators.Number(name="Listings", value=listings, **g_opts, format='{value:,.0f}')
    ban2 = pn.indicators.Number(name="Super Hosts", value=super_host, **g_opts, format='{value:,.0f}')
    ban3 = pn.indicators.Number(name="Avg Price/Night", value=avg_price, **g_opts, format='${value:,.0f}')
    
    return pn.Column(main_title, pn.Row(ban1, ban2, ban3, align='start'), align='start', height=80, width=150)

### Exercise

Create a BAN function for the mean price per bed type and have it interact with our nps score

1. Your function will take the nps widget
2. filter for an nps
3. that dataframe will be grouped by the bed_type column and
4. you will take the mean of the price column
5. display the ban and the widget in a column

### The Title

In [65]:
text = pn.pane.Markdown(
    f"# Airbnb Listings Analysis in Sydney", styles={"color": "#FFFFFF"}, height=50, sizing_mode="stretch_width", margin=(0,0,0,5)
)
text

Markdown(str, height=50, margin=(0, 0, 0, 5), sizing_mode='stretch_width', styles={'color': '#FFFFFF'})

In [66]:
img1 = pn.pane.PNG("https://i.pinimg.com/originals/a3/cd/30/a3cd30c0ba0e7f827dfe22e7a7011cd8.gif", height=50, sizing_mode="fixed", align="center")
img1

PNG(str, align='center', height=50, sizing_mode='fixed')

In [67]:
img2 = pn.pane.PNG("https://e7.pngegg.com/pngimages/388/581/png-clipart-sydney-opera-house-city-of-sydney-cartoon-illustration-sydney-opera-house-creative-cartoon-cartoon-character-angle.png", height=50, sizing_mode="fixed", align="center")
img2

PNG(str, align='center', height=50, sizing_mode='fixed')

In [68]:
title = pn.Row(text, pn.Spacer(), img1, img2, styles={'background': '#5F6062'}, sizing_mode='fixed', height=80, width=1050)
title

Row(height=80, sizing_mode='fixed', styles={'background': '#5F6062'}, width=1050)
    [0] Markdown(str, height=50, margin=(0, 0, 0, 5), sizing_mode='stretch_width', styles={'color': '#FFFFFF'})
    [1] Spacer()
    [2] PNG(str, align='center', height=50, sizing_mode='fixed')
    [3] PNG(str, align='center', height=50, sizing_mode='fixed')

### The Theme

We will use a slightly different theme from the static dashboard. Here is the [inspiration](https://bigbookofdashboards.com/images/Figure13.png) for this one.

In [69]:
from bokeh.themes.theme import Theme

theme = Theme(
    json={
    'attrs' : {
        'figure' : {
            'background_fill_color': '#5F6062',
            'border_fill_color': '#5F6062',
            'outline_line_color': '#5F6062',
        },
        'Grid': {
            'grid_line_dash': [6, 4],
            'grid_line_alpha': .3,
        },

        'Axis': {
            'major_label_text_color': '#D5E051',
            'axis_label_text_color': '#D5E051',
            'major_tick_line_color': '#D5E051',
            'minor_tick_line_color': '#D5E051',
            'axis_line_color': "#D5E051"
        },
        'Title': {
            'text_color': '#FFFFFF'
        }
    }
})

hv.renderer('bokeh').theme = theme

## 05. Putting it all Together

The most important piece of this part is the sizing of your dashboard or app. Something that can work well is 
to grab a pen and paper and draw what we envision as a dashboard before creating it. Or, we come back to every 
element while drawing boxes on a piece of paper and assigning the width and height to each line.

In [70]:
c1  = pn.Column(bans, pn.Spacer(height=20), get_rev_table, width=250, height=290)

In [71]:
c2 = pn.Column(pn.Row(p_type, nps, align='center'), cat_whisker, height=290, align='center')

In [72]:
r1 = pn.Row(c1, pn.Spacer(width=100), c2, sizing_mode='fixed', align='center', width=1000, height=350)

In [73]:
r2 = pn.Row(my_dots, get_map, align='center', sizing_mode='fixed', width=1100, height=420)

In [74]:
dashboard = pn.Column(
    title, r1, r2, styles={"background":'#5F6062'}, sizing_mode='fixed', align='center', height=950, width=1050
)

In [75]:
dashboard.show(threaded=True)

<StoppableThread(Thread-6 (get_server), started 140116865578560)>

Launching server at http://localhost:45083


In [ ]:
# dashboard.save('dashboards/interactive_dash.html')

### Exercise

Your task is to take any of the charts 

1. Create a Tabs element using `pn.Tabs`
2. Add the dashboard we created together to one tab
3. Add one of the elements you created, and a title to the other tab
4. Add an image of Python to the last tab

## 06. Summary

1. Widgets can be created for categories or discrete numbers and floats, here we have used mainly categories
2. Start building your visualisations step by step and once you have an MVP, focus onn wrapping the operations in functions
1. Using the tools chosen for the tutorial, interactive charts require functions that are tied to widgets
2. These functions get computed every time a value changes and the visual display gets updated
3. The larger the dataset the longer a computation might take